# Imports / Types

In [1]:
from collections import namedtuple, Counter, OrderedDict
from pprint import pprint

In [2]:
SpiFrame = namedtuple("SpiFrame", ["name", "ftype", "start_time", "duration", "mosi", "miso"])

# Functions

In [3]:
def parse_frames(lines: list[str]) -> list[SpiFrame]:
    frames: list[SpiFrame] = []

    for frame in lines[1:]:
        name, ftype, start_time, duration, mosi, miso = frame.split(",")
        frames.append(SpiFrame(name, ftype, start_time, duration, mosi, miso))
    return frames

def pack_4(frames: list[SpiFrame]) -> OrderedDict[str, str]:
    data: OrderedDict[str, str] = OrderedDict()

    i = 0
    while i < len(frames):
        frame = frames[i]
        if frame.ftype == '"result"' and i+3 < len(frames):
            data[frame.start_time] = f"{frame.miso} {frames[i+1].miso} {frames[i+2].miso} {frames[i+3].miso}"
            i += 3
        i += 1
    return data

def count_unique_frames(data: OrderedDict[str, str]):
    cnt = Counter()
    for _, miso in data.items():
        cnt[miso] += 1
    pprint(cnt)

# TODO: Make this a bit more general, so that hex, dec and bin can
#   be printed without having to call reverse
def reverse_bits(h: str) -> tuple[int, str, str]:
    reversed = bin(int(h, 16))[2:].zfill(8)[::-1]
    decimal = int(reversed, 2)
    hexadecimal = f"0x{decimal:02X}"
    return decimal, hexadecimal, reversed

def print_reversed(frames: list[str]):
    print("original data\t\treversed decimal\treversed hexadecimal\treversed binary")
    for frame in frames:
        packages = frame.split()
        decimals, hexadecimals, binaries = zip(*[reverse_bits(package) for package in packages])
        print(f"{frame}\t{' '.join(str(d) for d in decimals)}\t\t{' '.join(list(hexadecimals))}\t{' '.join(list(binaries))}")

def print_data_table(frames: list[str]):
    print("original data\t\tbinary encoding\t\t\t\tdecimal encoding")
    for frame in frames:
        hex_pieces = frame.split()
        frame_decimal = [int(h, 16) for h in hex_pieces]
        frame_binary = [bin(d)[2:].zfill(8) for d in frame_decimal]
        print(f"{frame}\t{' '.join(frame_binary)}\t{' '.join(str(d) for d in frame_decimal)}")

---

# Data Analysis

In [4]:
with open("data/button-2-drive-up.csv", "r", encoding="utf-8") as f:
    lines = f.read().split()

In [5]:
len(lines)

18505

In [6]:
lines[0]

'name,type,start_time,duration,"mosi","miso"'

In [7]:
frames = parse_frames(lines)
len(frames)

18504

In [8]:
frames

[SpiFrame(name='"SPI"', ftype='"enable"', start_time='0.004146276', duration='4e-09', mosi='', miso=''),
 SpiFrame(name='"SPI"', ftype='"result"', start_time='0.0041478', duration='0.000299952', mosi='0x00', miso='0xA5'),
 SpiFrame(name='"SPI"', ftype='"result"', start_time='0.004494628', duration='0.000288868', mosi='0x00', miso='0x85'),
 SpiFrame(name='"SPI"', ftype='"result"', start_time='0.004809908', duration='0.00025998', mosi='0x00', miso='0x00'),
 SpiFrame(name='"SPI"', ftype='"result"', start_time='0.005096304', duration='0.000284956', mosi='0x00', miso='0x7F'),
 SpiFrame(name='"SPI"', ftype='"disable"', start_time='0.005445488', duration='4e-09', mosi='', miso=''),
 SpiFrame(name='"SPI"', ftype='"enable"', start_time='0.009134644', duration='4e-09', mosi='', miso=''),
 SpiFrame(name='"SPI"', ftype='"result"', start_time='0.009136172', duration='0.00032598', mosi='0x00', miso='0xA5'),
 SpiFrame(name='"SPI"', ftype='"result"', start_time='0.009509036', duration='0.000288916', m

In [9]:
data = pack_4(frames)

In [10]:
set(len(d.split()) for d in data.values())  # Sanity check: Data only contains batches of 4 bytes

{4}

In [11]:
set(data.values())

{'0x5A 0x08 0x10 0x00',
 '0x5A 0x08 0x10 0x10',
 '0x5A 0x08 0x10 0x20',
 '0x5A 0x08 0x10 0x40',
 '0x5A 0x08 0x10 0x60',
 '0x5A 0x08 0x10 0x80',
 '0x5A 0x08 0x10 0x90',
 '0x5A 0x08 0x10 0xA0',
 '0x5A 0x08 0x10 0xC0',
 '0x5A 0x08 0x10 0xE0',
 '0x5A 0x08 0x90 0x00',
 '0x5A 0x08 0xE0 0x10',
 '0x5A 0x08 0xE0 0x20',
 '0x5A 0x08 0xE0 0x60',
 '0x5A 0x08 0xE0 0x90',
 '0x5A 0x08 0xE0 0xA0',
 '0x5A 0x08 0xE0 0xE0',
 '0xA5 0x85 0x00 0x7F',
 '0xA5 0x85 0x10 0x6F'}

In [12]:
count_unique_frames(data)

Counter({'0xA5 0x85 0x10 0x6F': 2428,
         '0xA5 0x85 0x00 0x7F': 639,
         '0x5A 0x08 0xE0 0x20': 1,
         '0x5A 0x08 0xE0 0xA0': 1,
         '0x5A 0x08 0xE0 0x60': 1,
         '0x5A 0x08 0xE0 0xE0': 1,
         '0x5A 0x08 0xE0 0x10': 1,
         '0x5A 0x08 0xE0 0x90': 1,
         '0x5A 0x08 0x10 0x00': 1,
         '0x5A 0x08 0x10 0x80': 1,
         '0x5A 0x08 0x10 0x40': 1,
         '0x5A 0x08 0x10 0xC0': 1,
         '0x5A 0x08 0x10 0x20': 1,
         '0x5A 0x08 0x10 0xA0': 1,
         '0x5A 0x08 0x10 0x60': 1,
         '0x5A 0x08 0x10 0xE0': 1,
         '0x5A 0x08 0x10 0x10': 1,
         '0x5A 0x08 0x10 0x90': 1,
         '0x5A 0x08 0x90 0x00': 1})


In [13]:
for start_time, miso in data.items():
    if miso.startswith("0x5A"):
        print(f"{start_time}\t{miso}")

2.62008163	0x5A 0x08 0xE0 0x20
3.17880782	0x5A 0x08 0xE0 0xA0
3.73790791	0x5A 0x08 0xE0 0x60
4.29666157	0x5A 0x08 0xE0 0xE0
4.85538271	0x5A 0x08 0xE0 0x10
5.41407471	0x5A 0x08 0xE0 0x90
5.97280619	0x5A 0x08 0x10 0x00
6.53153903	0x5A 0x08 0x10 0x80
7.09523752	0x5A 0x08 0x10 0x40
7.65396234	0x5A 0x08 0x10 0xC0
8.21271848	0x5A 0x08 0x10 0x20
8.77155664	0x5A 0x08 0x10 0xA0
9.33030042	0x5A 0x08 0x10 0x60
9.88902517	0x5A 0x08 0x10 0xE0
10.4477285	0x5A 0x08 0x10 0x10
11.0064762	0x5A 0x08 0x10 0x90
11.6547707	0x5A 0x08 0x90 0x00


In [14]:
print_reversed([miso for miso in data.values() if miso.startswith("0x5A")])

original data		reversed decimal	reversed hexadecimal	reversed binary
0x5A 0x08 0xE0 0x20	90 16 7 4		0x5A 0x10 0x07 0x04	01011010 00010000 00000111 00000100
0x5A 0x08 0xE0 0xA0	90 16 7 5		0x5A 0x10 0x07 0x05	01011010 00010000 00000111 00000101
0x5A 0x08 0xE0 0x60	90 16 7 6		0x5A 0x10 0x07 0x06	01011010 00010000 00000111 00000110
0x5A 0x08 0xE0 0xE0	90 16 7 7		0x5A 0x10 0x07 0x07	01011010 00010000 00000111 00000111
0x5A 0x08 0xE0 0x10	90 16 7 8		0x5A 0x10 0x07 0x08	01011010 00010000 00000111 00001000
0x5A 0x08 0xE0 0x90	90 16 7 9		0x5A 0x10 0x07 0x09	01011010 00010000 00000111 00001001
0x5A 0x08 0x10 0x00	90 16 8 0		0x5A 0x10 0x08 0x00	01011010 00010000 00001000 00000000
0x5A 0x08 0x10 0x80	90 16 8 1		0x5A 0x10 0x08 0x01	01011010 00010000 00001000 00000001
0x5A 0x08 0x10 0x40	90 16 8 2		0x5A 0x10 0x08 0x02	01011010 00010000 00001000 00000010
0x5A 0x08 0x10 0xC0	90 16 8 3		0x5A 0x10 0x08 0x03	01011010 00010000 00001000 00000011
0x5A 0x08 0x10 0x20	90 16 8 4		0x5A 0x10 0x08 0x04	01011010 0

In [15]:
with open("data/button-2-drive-down.csv", "r", encoding="utf-8") as f:
    lines_down = f.read().split()

frames_down = parse_frames(lines_down)
data_down = pack_4(frames_down)

set(len(d.split()) for d in data_down.values())  # Sanity check: Data only contains batches of 4 bytes

{4}

In [16]:
count_unique_frames(data_down)

Counter({'0xA5 0x85 0x10 0x6F': 1329,
         '0xA5 0x85 0x00 0x7F': 326,
         '0x5A 0x08 0x90 0xE0': 1,
         '0x5A 0x08 0x90 0x60': 1,
         '0x5A 0x08 0x90 0xA0': 1,
         '0x5A 0x08 0x90 0x20': 1,
         '0x5A 0x08 0x90 0xC0': 1,
         '0x5A 0x08 0x90 0x40': 1,
         '0x5A 0x08 0x90 0x80': 1,
         '0x5A 0x08 0x90 0x00': 1,
         '0x00 0x00 0x00 0x00': 1,
         '0x00 0x80 0x00 0x07': 1})


In [17]:
for start_time, miso in data_down.items():
    if miso.startswith("0x5A"):
        print(f"{start_time}\t{miso}")

1.27949046	0x5A 0x08 0x90 0xE0
2.73107526	0x5A 0x08 0x90 0x60
3.37939679	0x5A 0x08 0x90 0xA0
3.93826862	0x5A 0x08 0x90 0x20
4.49684859	0x5A 0x08 0x90 0xC0
5.05548906	0x5A 0x08 0x90 0x40
5.61413758	0x5A 0x08 0x90 0x80
6.20252918	0x5A 0x08 0x90 0x00


In [18]:
print_reversed([miso for miso in data_down.values() if miso.startswith("0x5A")])

original data		reversed decimal	reversed hexadecimal	reversed binary
0x5A 0x08 0x90 0xE0	90 16 9 7		0x5A 0x10 0x09 0x07	01011010 00010000 00001001 00000111
0x5A 0x08 0x90 0x60	90 16 9 6		0x5A 0x10 0x09 0x06	01011010 00010000 00001001 00000110
0x5A 0x08 0x90 0xA0	90 16 9 5		0x5A 0x10 0x09 0x05	01011010 00010000 00001001 00000101
0x5A 0x08 0x90 0x20	90 16 9 4		0x5A 0x10 0x09 0x04	01011010 00010000 00001001 00000100
0x5A 0x08 0x90 0xC0	90 16 9 3		0x5A 0x10 0x09 0x03	01011010 00010000 00001001 00000011
0x5A 0x08 0x90 0x40	90 16 9 2		0x5A 0x10 0x09 0x02	01011010 00010000 00001001 00000010
0x5A 0x08 0x90 0x80	90 16 9 1		0x5A 0x10 0x09 0x01	01011010 00010000 00001001 00000001
0x5A 0x08 0x90 0x00	90 16 9 0		0x5A 0x10 0x09 0x00	01011010 00010000 00001001 00000000


**Note: Up until know I've had the logic analyzer in the wrong setting (MSB first, while LSB first would have been the write setting). That's why the following data might look a bit different.**

### Analyzing Button-Codes

This is the order of the buttons on the control-panel from left to right:

1. Button 1
2. Button 2
3. Button 3
4. Button 4
5. Button S
6. Button UP
7. Button DOWN

Their is kind of a binary pattern in the encoding of the button-data. However the order is switched up from the representation on the control-panel:

In [19]:
print_data_table([
    "0xA5 0xA1 0x00 0xFE",  # Idle
    "0xA5 0xA1 0x02 0xFC",  # Button 4
    "0xA5 0xA1 0x04 0xFA",  # Button 1
    "0xA5 0xA1 0x08 0xF6",  # Button 2
    "0xA5 0xA1 0x10 0xEE",  # Button 3
    "0xA5 0xA1 0x20 0xDE",  # Button S
    "0xA5 0xA1 0x40 0xBE",  # Button UP
    "0xA5 0xA1 0x80 0x7E",  # Button DOWN
])

original data		binary encoding				decimal encoding
0xA5 0xA1 0x00 0xFE	10100101 10100001 00000000 11111110	165 161 0 254
0xA5 0xA1 0x02 0xFC	10100101 10100001 00000010 11111100	165 161 2 252
0xA5 0xA1 0x04 0xFA	10100101 10100001 00000100 11111010	165 161 4 250
0xA5 0xA1 0x08 0xF6	10100101 10100001 00001000 11110110	165 161 8 246
0xA5 0xA1 0x10 0xEE	10100101 10100001 00010000 11101110	165 161 16 238
0xA5 0xA1 0x20 0xDE	10100101 10100001 00100000 11011110	165 161 32 222
0xA5 0xA1 0x40 0xBE	10100101 10100001 01000000 10111110	165 161 64 190
0xA5 0xA1 0x80 0x7E	10100101 10100001 10000000 01111110	165 161 128 126


## Analyzing a longer recording

Driving to standing position and back to sitting position

In [20]:
with open("data/drive-to-2-drive-to-1.csv", "r", encoding="utf-8") as f:
    lines_up_down = f.read().split()

frames_up_down = parse_frames(lines_up_down)
data_up_down = pack_4(frames_up_down)

set(len(d.split()) for d in data_up_down.values())  # Sanity check: Data only contains batches of 4 bytes

{4}

In [21]:
count_unique_frames(data_up_down)

Counter({'0xA5 0xA1 0x00 0xFE': 4599,
         '0xA5 0xA1 0x04 0xFA': 3332,
         '0xA5 0xA1 0x08 0xF6': 2503,
         '0x5A 0x10 0x07 0x03': 2,
         '0x5A 0x10 0x07 0x04': 2,
         '0x5A 0x10 0x07 0x05': 2,
         '0x5A 0x10 0x07 0x06': 2,
         '0x5A 0x10 0x07 0x07': 2,
         '0x5A 0x10 0x07 0x08': 2,
         '0x5A 0x10 0x07 0x09': 2,
         '0x5A 0x10 0x08 0x00': 2,
         '0x5A 0x10 0x08 0x01': 2,
         '0x5A 0x10 0x08 0x02': 2,
         '0x5A 0x10 0x08 0x03': 2,
         '0x5A 0x10 0x08 0x04': 2,
         '0x5A 0x10 0x08 0x05': 2,
         '0x5A 0x10 0x08 0x06': 2,
         '0x5A 0x10 0x08 0x07': 2,
         '0x5A 0x10 0x08 0x08': 2,
         '0x5A 0x10 0x08 0x09': 2,
         '0x5A 0x10 0x09 0x00': 1,
         '0x5A 0x10 0x10 0x10': 1})


In [22]:
for start_time, miso in data_up_down.items():
    if miso.startswith("0x5A"):
        print(f"{start_time}\t{miso}")

1.11718302	0x5A 0x10 0x07 0x03
2.82843954	0x5A 0x10 0x07 0x04
3.3871538	0x5A 0x10 0x07 0x05
3.94628183	0x5A 0x10 0x07 0x06
4.50505025	0x5A 0x10 0x07 0x07
5.06380986	0x5A 0x10 0x07 0x08
5.62260277	0x5A 0x10 0x07 0x09
6.18140317	0x5A 0x10 0x08 0x00
6.7402282	0x5A 0x10 0x08 0x01
7.29903691	0x5A 0x10 0x08 0x02
7.85784968	0x5A 0x10 0x08 0x03
8.41669991	0x5A 0x10 0x08 0x04
8.97550896	0x5A 0x10 0x08 0x05
9.53435949	0x5A 0x10 0x08 0x06
10.093176	0x5A 0x10 0x08 0x07
10.6569637	0x5A 0x10 0x08 0x08
11.2157609	0x5A 0x10 0x08 0x09
11.864031	0x5A 0x10 0x09 0x00
17.2727823	0x5A 0x10 0x08 0x09
17.8316946	0x5A 0x10 0x08 0x08
18.3908749	0x5A 0x10 0x08 0x07
18.9497777	0x5A 0x10 0x08 0x06
19.5086127	0x5A 0x10 0x08 0x05
20.0674673	0x5A 0x10 0x08 0x04
20.6263135	0x5A 0x10 0x08 0x03
21.1851855	0x5A 0x10 0x08 0x02
21.7440361	0x5A 0x10 0x08 0x01
22.3028829	0x5A 0x10 0x08 0x00
22.8617178	0x5A 0x10 0x07 0x09
23.4205638	0x5A 0x10 0x07 0x08
23.9794343	0x5A 0x10 0x07 0x07
24.5432703	0x5A 0x10 0x07 0x06
25.1021338	0

In [23]:
print_data_table([miso for miso in data_up_down.values() if miso.startswith("0x5A")])

original data		binary encoding				decimal encoding
0x5A 0x10 0x07 0x03	01011010 00010000 00000111 00000011	90 16 7 3
0x5A 0x10 0x07 0x04	01011010 00010000 00000111 00000100	90 16 7 4
0x5A 0x10 0x07 0x05	01011010 00010000 00000111 00000101	90 16 7 5
0x5A 0x10 0x07 0x06	01011010 00010000 00000111 00000110	90 16 7 6
0x5A 0x10 0x07 0x07	01011010 00010000 00000111 00000111	90 16 7 7
0x5A 0x10 0x07 0x08	01011010 00010000 00000111 00001000	90 16 7 8
0x5A 0x10 0x07 0x09	01011010 00010000 00000111 00001001	90 16 7 9
0x5A 0x10 0x08 0x00	01011010 00010000 00001000 00000000	90 16 8 0
0x5A 0x10 0x08 0x01	01011010 00010000 00001000 00000001	90 16 8 1
0x5A 0x10 0x08 0x02	01011010 00010000 00001000 00000010	90 16 8 2
0x5A 0x10 0x08 0x03	01011010 00010000 00001000 00000011	90 16 8 3
0x5A 0x10 0x08 0x04	01011010 00010000 00001000 00000100	90 16 8 4
0x5A 0x10 0x08 0x05	01011010 00010000 00001000 00000101	90 16 8 5
0x5A 0x10 0x08 0x06	01011010 00010000 00001000 00000110	90 16 8 6
0x5A 0x10 0x08 0x07	01011

No big surprises here. Only the "go to sleep" frame